# Data Organization Script # 

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import pi
# import seaborn as sns
# import matplotlib as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../Final"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


Data_Organization.ipynb
Final_Data_Organization.ipynb
FinalMarchMadnessAdvStats.csv
Final MLP.ipynb
MarchMadnessAdvStats.csv
OldWNCAATourneyCompactResults.csv
OldWNCAATourneySeeds.csv
wModel.py
WNCAATourneyCompactResults.csv
WNCAATourneyDetailedResults.csv
WNCAATourneySeeds.csv
WNCAATourneySlots.csv
WRegularSeasonCompactResults.csv
WRegularSeasonDetailedResults.csv
WSampleSubmissionStage2.csv
WSeasons.csv
WTeams.csv
WTeamSpellings.csv



# Load and Organize Training data


In [33]:
data_dir = '../Final/'
df_seeds = pd.read_csv(data_dir + 'OldWNCAATourneySeeds.csv')
df_tour = pd.read_csv(data_dir + 'OldWNCAATourneyCompactResults.csv')
df_tour.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,137,3104,94,3422,46,H,0
1,1998,137,3112,75,3365,63,H,0
2,1998,137,3163,93,3193,52,H,0
3,1998,137,3198,59,3266,45,H,0
4,1998,137,3203,74,3208,72,A,0


Add advanced stats from https://www.kaggle.com/lnatml/feature-engineering-with-advanced-stats/notebook

In [34]:
df = pd.read_csv(data_dir+'WRegularSeasonDetailedResults.csv')


In [35]:
#Points Winning/Losing Team
df['WPts'] = df.apply(lambda row: 2*(row.WFGM-row.WFGM3) + 3*row.WFGM3 + row.WFTM, axis=1)
df['LPts'] = df.apply(lambda row: 2*(row.LFGM-row.WFGM3) + 3*row.LFGM3 + row.LFTM, axis=1)

#Calculate Winning/losing Team Possesion Feature
wPos = df.apply(lambda row: 0.96*(row.WFGA + row.WTO + 0.44*row.WFTA - row.WOR), axis=1)
lPos = df.apply(lambda row: 0.96*(row.LFGA + row.LTO + 0.44*row.LFTA - row.LOR), axis=1)
#two teams use almost the same number of possessions in a game
#(plus/minus one or two - depending on how quarters end)
#so let's just take the average
df['Pos'] = (wPos+lPos)/2

#Offensive efficiency (OffRtg) = 100 x (Points / Possessions)
df['WOffRtg'] = df.apply(lambda row: 100 * (row.WPts / row.Pos), axis=1)
df['LOffRtg'] = df.apply(lambda row: 100 * (row.LPts / row.Pos), axis=1)
#Defensive efficiency (DefRtg) = 100 x (Opponent points / Opponent possessions)
df['WDefRtg'] = df.LOffRtg
df['LDefRtg'] = df.WOffRtg
#Net Rating = Off.eff - Def.eff
df['WNetRtg'] = df.apply(lambda row:(row.WOffRtg - row.LDefRtg), axis=1)
df['LNetRtg'] = df.apply(lambda row:(row.LOffRtg - row.LDefRtg), axis=1)
                         
#Assist Ratio : Percentage of team possessions that end in assists
df['WAstR'] = df.apply(lambda row: 100 * row.WAst / (row.WFGA + 0.44*row.WFTA + row.WAst + row.WTO), axis=1)
df['LAstR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
#Turnover Ratio: Number of turnovers of a team per 100 possessions used.
#(TO * 100) / (FGA + (FTA * 0.44) + AST + TO
df['WTOR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
df['LTOR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
                    
#The Shooting Percentage : Measure of Shooting Efficiency (FGA/FGA3, FTA)
df['WTSP'] = df.apply(lambda row: 100 * row.WPts / (2 * (row.WFGA + 0.44 * row.WFTA)), axis=1)
df['LTSP'] = df.apply(lambda row: 100 * row.LPts / (2 * (row.LFGA + 0.44 * row.LFTA)), axis=1)
#eFG% : Effective Field Goal Percentage adjusting for the fact that 3pt shots are more valuable 
df['WeFGP'] = df.apply(lambda row:(row.WFGM + 0.5 * row.WFGM3) / row.WFGA, axis=1)      
df['LeFGP'] = df.apply(lambda row:(row.LFGM + 0.5 * row.LFGM3) / row.LFGA, axis=1)   
#FTA Rate : How good a team is at drawing fouls.
df['WFTAR'] = df.apply(lambda row: row.WFTA / row.WFGA, axis=1)
df['LFTAR'] = df.apply(lambda row: row.LFTA / row.LFGA, axis=1)
                         
#OREB% : Percentage of team offensive rebounds
df['WORP'] = df.apply(lambda row: row.WOR / (row.WOR + row.LDR), axis=1)
df['LORP'] = df.apply(lambda row: row.LOR / (row.LOR + row.WDR), axis=1)
#DREB% : Percentage of team defensive rebounds
df['WDRP'] = df.apply(lambda row: row.WDR / (row.WDR + row.LOR), axis=1)
df['LDRP'] = df.apply(lambda row: row.LDR / (row.LDR + row.WOR), axis=1)                                      
#REB% : Percentage of team total rebounds
df['WRP'] = df.apply(lambda row: (row.WDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1)
df['LRP'] = df.apply(lambda row: (row.LDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1)

After creating adv stats, now we need to split the winning and losing team stats for a more overall picture

In [36]:
df_TeamID = pd.concat((df['WTeamID'], df['LTeamID']), axis=1)
df_adv2 = df.iloc[:, 34:].copy()
df_adv = pd.concat((df_TeamID, df_adv2), axis=1)
df_adv = pd.concat((df['Season'], df_adv), axis=1)
# df_adv.head()
names = df_adv.columns.values 
print(names)


['Season' 'WTeamID' 'LTeamID' 'WPts' 'LPts' 'Pos' 'WOffRtg' 'LOffRtg'
 'WDefRtg' 'LDefRtg' 'WNetRtg' 'LNetRtg' 'WAstR' 'LAstR' 'WTOR' 'LTOR'
 'WTSP' 'LTSP' 'WeFGP' 'LeFGP' 'WFTAR' 'LFTAR' 'WORP' 'LORP' 'WDRP' 'LDRP'
 'WRP' 'LRP']


In [37]:
Wnames = ['Season', 'WTeamID','WPts','Pos', 'WOffRtg' ,'WDefRtg',
          'WNetRtg', 'WAstR', 'WTOR', 'WTSP','WeFGP','WFTAR', 'WORP', 'WDRP','WRP']
Lnames = ['Season', 'LTeamID', 'LPts', 'Pos', 'LOffRtg','LDefRtg',
          'LNetRtg', 'LAstR', 'LTOR','LTSP', 'LeFGP','LFTAR', 'LORP', 'LDRP', 'LRP' ]
df_advW = df_adv.loc[:,Wnames].copy()
df_advL = df_adv.loc[:,Lnames].copy()
df_advW.rename(columns={'WTeamID':'TeamID'}, inplace=True)
df_advL.rename(columns={'LTeamID':'TeamID'}, inplace=True)

Must concat then group the advanced stats to get season averages for each team

In [51]:
names = ['Season', 'TeamID', 'Pts', 'Pos', 'OffRtg','DefRtg',
          'NetRtg', 'AstR', 'TOR','TSP', 'eFGP','FTAR', 'ORP', 'DRP', 'RP' ]
df_advL.columns = names
df_advW.columns = names
df_A = pd.concat((df_advL, df_advW), axis=0, ignore_index=True)
groupedA = df_A.groupby(['Season', 'TeamID'], as_index=False)
df_A = groupedA.agg(np.mean)
# df_A.shape
df_advL = df_A.copy()
df_advW = df_A.copy()


,Season,TeamID,Pts,Pos,OffRtg,DefRtg,NetRtg,AstR,TOR,TSP,eFGP,FTAR,ORP,DRP,RP
3113,2018,3460,70.548387,66.005265,106.589342,93.116864,-4.698713,12.816905,11.289681,48.246051,0.440026,0.328681,0.393259,0.699035,0.507456
3114,2018,3461,61.310345,59.802041,102.791828,89.322779,-4.766990,15.955198,11.579984,53.411961,0.487196,0.307635,0.271120,0.746692,0.516928
3115,2018,3462,56.833333,63.474560,89.336018,99.979243,-19.984408,13.789852,13.103158,48.409972,0.467221,0.299154,0.254764,0.690159,0.506806
3116,2018,3463,64.928571,70.439314,92.077836,90.863108,-10.824932,12.735682,12.382035,46.392436,0.448651,0.287338,0.297958,0.694608,0.492131
3117,2018,3464,63.862069,64.343172,98.964329,95.303133,-9.654426,14.956915,13.176745,52.445229,0.479859,0.255965,0.255406,0.675553,0.510688


In [39]:
df_Adv= df_A[df_A.Season<2018]
df_Adv.tail()

,Season,TeamID,Pts,Pos,OffRtg,DefRtg,NetRtg,AstR,TOR,TSP,eFGP,FTAR,ORP,DRP,RP
2764,2017,3460,66.833333,65.901440,101.458107,94.199399,-6.155488,11.759383,12.002533,47.092966,0.427063,0.327959,0.342786,0.691388,0.496938
2765,2017,3461,64.678571,60.790629,106.321271,83.375404,-2.751090,17.119476,11.449986,55.035055,0.507625,0.281937,0.278874,0.732086,0.529966
2766,2017,3462,60.266667,65.955200,91.326938,94.694173,-14.847354,13.817762,12.631707,47.183942,0.452031,0.302004,0.275644,0.699656,0.532335
2767,2017,3463,61.370370,66.436267,92.523655,87.589066,-9.146557,11.723753,10.996659,46.928161,0.432859,0.305165,0.286041,0.715887,0.493744
2768,2017,3464,68.142857,69.507429,98.161594,105.665112,-13.451377,15.377787,14.064156,51.488821,0.460832,0.237699,0.228067,0.662101,0.478820


Cut off the region identifier from the seed number

In [40]:
def seed_to_int(seed):
    #Get just the digits from the seeding. Return as int
    s_int = int(seed[1:3])
    return s_int
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(columns=['Seed'], inplace=True) # This is the string label

df_tour.drop(labels=['DayNum', 'WScore', 'LScore','WLoc', 'NumOT'], inplace=True, axis=1)
# df_tour.head()

Merge the Seeds with their corresponding TeamIDs in the compact results dataframe.

In [41]:
df_winSeeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossSeeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})


df_dummy = pd.merge(left=df_tour, right=df_lossSeeds, how='left', on=['Season', 'LTeamID'])

df_concat = pd.merge(left=df_dummy, right=df_winSeeds, how='left' ,on=['Season', 'WTeamID'])
# df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed
# df_concat.head()

 Make a winner and loser DF with seed, relative seed, and win/loss. 

In [53]:
df_wins = pd.DataFrame()
df_wins['Seed'] = df_concat['WSeed']
# df_wins['SeedDiff'] = df_concat['SeedDiff']
df_wins['TeamID'] = df_concat['WTeamID']
df_wins['Season'] = df_concat['Season']
df_wins['Result'] = 1
df_wins = df_wins[df_wins.Season>=2010]
df_wins = pd.merge(left=df_wins, right=df_advW, how='left', on=['Season', 'TeamID'])
# df_wins.tail()

In [55]:
df_losses = pd.DataFrame()
df_losses['Seed'] = df_concat['LSeed']
# df_losses['SeedDiff'] = -df_concat['SeedDiff']
df_losses['TeamID'] = df_concat['LTeamID']
df_losses['Season'] = df_concat['Season']
df_losses['Result'] = 0
df_losses = df_losses[df_losses.Season>=2010]
df_losses = pd.merge(left=df_losses, right=df_advL, how='left', on=['Season', 'TeamID'])
# df_losses.head()

Prefix the cols with 'Opp' in dummy frames to allow for horizontal concat in order to expand data to be

| winr_stat    losr_stat |
                     
| losr_stat    winr_stat |

In [57]:
df_lossesOpp = df_losses.copy()

df_lossesOpp.drop(labels=['Season', 'Result'], inplace=True, axis=1)
new_names = [(i,'Opp'+i) for i in df_lossesOpp.columns.values]
df_lossesOpp.rename(columns = dict(new_names), inplace=True)
# df_lossesOpp.isnull().any()

In [59]:
df_winsOpp = df_wins.copy()

df_winsOpp.drop(labels=['Season', 'Result'], inplace=True, axis=1)
new_names = [(i,'Opp'+i) for i in df_winsOpp.columns.values]
df_winsOpp.rename(columns = dict(new_names), inplace=True)
# df_winsOpp.head()

In [60]:
df_winloss = pd.concat([df_wins, df_lossesOpp], axis=1)
df_losswin = pd.concat([df_losses, df_winsOpp], axis=1)

Divide train and test case

In [61]:

df_winlossTest = df_winloss[df_winloss.Season>2014]
df_losswinTest = df_losswin[df_losswin.Season>2014]


In [62]:
df_winloss = df_winloss[df_winloss.Season<2014]
df_losswin = df_losswin[df_losswin.Season<2014]

# Combine into final dataframe

In [63]:
df_finalData = pd.concat((df_winloss, df_losswin))
results = df_finalData['Result']
df_finalData.drop(labels=['Result'], inplace=True, axis=1)
df_finalData.insert(0, 'Result', results)
df_finalData.shape

(504, 32)

Remove team ID and season -> cannot be scaled and not a metric. Dummy 

*** Also cut data after 2014 so the model isnt trained on test data***

In [64]:
# df_finalData = df_finalData[df_finalData<2014]
df_finalData.drop(labels=['TeamID', 'Season', 'OppTeamID'], inplace=True, axis=1)
# df_finalData.head()

# Combine into test dataframe

In [65]:
df_finalDataTest = pd.concat((df_winlossTest, df_losswinTest))
results = df_finalDataTest['Result']
df_finalDataTest.drop(labels=['Result'], inplace=True, axis=1)
df_finalDataTest.insert(0, 'Result', results)
df_finalDataTest.shape

(378, 32)

In [66]:
df_finalDataTest.drop(labels=['TeamID', 'Season', 'OppTeamID'], inplace=True, axis=1)
# df_finalData.head()

In [67]:
df_finalData.isnull().any()

Result       False
Seed         False
Pts          False
Pos          False
OffRtg       False
DefRtg       False
NetRtg       False
AstR         False
TOR          False
TSP          False
eFGP         False
FTAR         False
ORP          False
DRP          False
RP           False
OppSeed      False
OppPts       False
OppPos       False
OppOffRtg    False
OppDefRtg    False
OppNetRtg    False
OppAstR      False
OppTOR       False
OppTSP       False
OppeFGP      False
OppFTAR      False
OppORP       False
OppDRP       False
OppRP        False
dtype: bool

In [68]:
df_finalData.to_csv('MarchMadnessFeatures.csv', index=False)

In [69]:
df_finalDataTest.head()

,Result,Seed,Pts,Pos,OffRtg,DefRtg,NetRtg,AstR,TOR,TSP,...,OppDefRtg,OppNetRtg,OppAstR,OppTOR,OppTSP,OppeFGP,OppFTAR,OppORP,OppDRP,OppRP
315,1,10,60.900000,62.540800,97.201890,86.525417,-6.424856,12.954466,10.179447,46.912114,...,89.450628,-4.486997,16.686535,13.553815,51.023183,0.476951,0.308167,0.286657,0.648582,0.484199
316,1,2,79.151515,69.635491,113.451728,91.972182,-2.155844,18.804983,13.626572,53.841813,...,98.674588,-19.119430,12.420589,12.098924,46.066267,0.439636,0.340404,0.252468,0.628800,0.468162
317,1,4,67.843750,68.841600,97.879023,89.821181,-7.302903,16.033422,11.498919,49.331381,...,82.561987,-2.316106,14.812893,10.349721,51.245087,0.474557,0.241561,0.347719,0.674043,0.508858
318,1,7,75.838710,69.889239,109.127255,87.313470,-2.509395,14.156045,11.441876,53.798529,...,88.883606,-5.561061,16.457164,13.678371,52.611981,0.480131,0.310966,0.307847,0.705065,0.519568
319,1,9,87.575758,75.760291,115.520878,78.820007,-2.027807,17.283513,12.823624,53.280106,...,100.119766,-7.512425,16.407623,16.401570,50.804305,0.478643,0.303378,0.338712,0.668999,0.518240


In [70]:
df_finalDataTest.to_csv('MarchMadnessTest.csv', index=False)

In [32]:
df_diffData.to_csv('MarchMadnessFeatureDifferences.csv', index=False)

In [33]:
df_Adv.to_csv('MarchMadnessAdvStats.csv', index=False)